# Imports & Functions

In [10]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os

import random
from math import floor
from matplotlib.pyplot import figure, show
from sklearn.model_selection import train_test_split
import os
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Data Preprocessing

Katrina Yu

In [11]:
path = "./data/"

df = pd.read_csv(path + "GOOG.csv")

print(df.shape)
print(df[0:5])

# Dropping date and Adj Close, not needed
df.drop(columns={"Date", "Adj Close"}, inplace=True)

out_close = df[["Close"]]

# Close Feature for Input
df.rename(columns={"Close": "In Close"}, inplace=True)
df = pd.concat([df, out_close], axis=1)

# Close Feature for Output
df.rename(columns={"Close": "Out Close"}, inplace=True)

print(df.shape)
print(df[0:5])

# Close Feature for Input
print("In Close:\n")
print(df[["In Close"][0:5]])

# Close Feature for Output
print("Out Close:\n")
print(df[["Out Close"][0:5]])

print(df.head())

(3578, 7)
         Date       Open       High        Low      Close  Adj Close    Volume
0  2004-08-19  49.676899  51.693783  47.669952  49.845802  49.845802  44994500
1  2004-08-20  50.178635  54.187561  49.925285  53.805050  53.805050  23005800
2  2004-08-23  55.017166  56.373344  54.172661  54.346527  54.346527  18393200
3  2004-08-24  55.260582  55.439419  51.450363  52.096165  52.096165  15361800
4  2004-08-25  52.140873  53.651051  51.604362  52.657513  52.657513   9257400
(3578, 6)
        Open       High        Low   In Close    Volume  Out Close
0  49.676899  51.693783  47.669952  49.845802  44994500  49.845802
1  50.178635  54.187561  49.925285  53.805050  23005800  53.805050
2  55.017166  56.373344  54.172661  54.346527  18393200  54.346527
3  55.260582  55.439419  51.450363  52.096165  15361800  52.096165
4  52.140873  53.651051  51.604362  52.657513   9257400  52.657513
In Close:

         In Close
0       49.845802
1       53.805050
2       54.346527
3       52.096165
4  

### Normalizing Data
Katrina Yu

In [12]:
print(df.head())

# Numeric Features
numeric_feats = ["Open", "High", "Low", "In Close", "Volume"]

# Data Normalization for Input Features, so exclude "Out Close"
for i in numeric_feats:
    encode_numeric_zscore(df, i)
    
print(df.shape)
print(df.head())

        Open       High        Low   In Close    Volume  Out Close
0  49.676899  51.693783  47.669952  49.845802  44994500  49.845802
1  50.178635  54.187561  49.925285  53.805050  23005800  53.805050
2  55.017166  56.373344  54.172661  54.346527  18393200  54.346527
3  55.260582  55.439419  51.450363  52.096165  15361800  52.096165
4  52.140873  53.651051  51.604362  52.657513   9257400  52.657513
(3578, 6)
       Open      High       Low  In Close    Volume  Out Close
0 -1.334844 -1.331443 -1.337457 -1.333645  4.533247  49.845802
1 -1.333101 -1.322843 -1.329562 -1.319893  1.870138  53.805050
2 -1.316291 -1.315305 -1.314693 -1.318012  1.311494  54.346527
3 -1.315446 -1.318526 -1.324223 -1.325828  0.944353  52.096165
4 -1.326284 -1.324693 -1.323684 -1.323879  0.205033  52.657513


### Train-Test Split for Training & Testing
Katrina Yu

In [13]:
'''
x,y = to_xy(df, "Out Close")

print(x, x.shape)
print(y, y.shape) 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

print("shape: \n")
print(df["Open"].shape[0])
'''
# Splitting 70% of data for Training & 30% for Testing
split_index = floor((df["Open"].shape[0])*0.70)

df_train = df[0:split_index]
df_test = df[split_index:]

print(df_train.shape)
print(df_test.shape)

(2504, 6)
(1074, 6)


### Sequence data to create x and y in the format RNN likes.
Katrina Yu

In [14]:
import numpy as np

def to_sequences(seq_size, data):
    x = []
    y = []

    for i in range(len(data)-seq_size-1):
        #print(i)
        window = data.iloc[i:(i+seq_size)][["Open", "High", "Low", "In Close", "Volume"]].values
        after_window = data.iloc[i+seq_size]["Out Close"]
        #window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)

In [15]:
print(len(df_train))

# Past 7 days used for prediction
SEQUENCE_SIZE = 7
x_train,y_train = to_sequences(SEQUENCE_SIZE, df_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE, df_test)

print("Shape of x_train: {}".format(x_train.shape))
print("Shape of x_test: {}".format(x_test.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of y_test: {}".format(y_test.shape))

#print("x_train:\n {}".format(x_train[0:3]))

#print("y_train:\n {}".format(y_train[0:3]))

2504
Shape of x_train: (2496, 7, 5)
Shape of x_test: (1066, 7, 5)
Shape of y_train: (2496,)
Shape of y_test: (1066,)


# Building RNN Model using LSTM
Katrina Yu

In [ ]:
model_dir = "models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

for i in range(3):
    model = Sequential()
    
    model.add(LSTM(128, input_shape=(SEQUENCE_SIZE,5), activation="relu"))
    #model.add(Dropout(0.1))
    model.add(Dense(64))
    #model.add(Dropout(0.1))
    #model.add(Dense(32))
    model.add(Dense(1))

    model.compile(loss="mean_squared_error", optimizer="adam")

    monitor = EarlyStopping(monitor="val_loss", min_delta=1e-3, patience=5, verbose=1, mode="auto")
    checkpointer = ModelCheckpoint(filepath="models/lstm_checkpoint.keras", verbose=0, save_best_only=True)

    model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor, checkpointer], verbose=2, epochs=100)  
    
model.load_weights('models/lstm_checkpoint.keras')

Epoch 1/100
78/78 - 2s - loss: 49750.3633 - val_loss: 461699.5000 - 2s/epoch - 23ms/step
Epoch 2/100
78/78 - 1s - loss: 2585.7500 - val_loss: 301394.0938 - 550ms/epoch - 7ms/step
Epoch 3/100
78/78 - 1s - loss: 618.6409 - val_loss: 214587.2656 - 521ms/epoch - 7ms/step
Epoch 4/100
78/78 - 0s - loss: 297.8044 - val_loss: 161553.1719 - 476ms/epoch - 6ms/step
Epoch 5/100
78/78 - 0s - loss: 200.2098 - val_loss: 107346.1406 - 483ms/epoch - 6ms/step
Epoch 6/100
78/78 - 0s - loss: 196.6184 - val_loss: 155087.0469 - 428ms/epoch - 5ms/step
Epoch 7/100
78/78 - 0s - loss: 170.2458 - val_loss: 76032.9844 - 491ms/epoch - 6ms/step
Epoch 8/100
78/78 - 0s - loss: 144.1667 - val_loss: 96674.4297 - 408ms/epoch - 5ms/step
Epoch 9/100
78/78 - 0s - loss: 121.7305 - val_loss: 103289.2734 - 407ms/epoch - 5ms/step
Epoch 10/100
78/78 - 0s - loss: 132.8250 - val_loss: 90307.9219 - 422ms/epoch - 5ms/step
Epoch 11/100
78/78 - 0s - loss: 146.5761 - val_loss: 58378.8750 - 444ms/epoch - 6ms/step
Epoch 12/100
78/78 - 0

In [ ]:
pred = model.predict(x_test)

print(pred.shape, y_test.shape)

score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))
print("R2:",metrics.r2_score(y_test,pred))
print("MSE:", metrics.mean_squared_error(y_test, pred))

34/34 [==============================] - 1s 2ms/step
(1066, 1) (1066,)
Score (RMSE): 123.04219706133296
R2: 0.6597219444047588
MSE: 15139.382257679894


In [ ]:
results = []
step = 2
j = 1
while j < 200:
    print(j)
    SEQUENCE_SIZE = j
    fileName= "models/lstm_checkpoint_" + str(j) + ".keras"
    x_train,y_train = to_sequences(SEQUENCE_SIZE, df_train)
    x_test,y_test = to_sequences(SEQUENCE_SIZE, df_test)

    model_dir = "models"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    for i in range(3):
        model = Sequential()
        
        model.add(LSTM(128, input_shape=(SEQUENCE_SIZE,5), activation="relu"))
        #model.add(Dropout(0.1))
        model.add(Dense(64))
        model.add(Dropout(0.1))
        #model.add(Dense(32))
        model.add(Dense(1))

        model.compile(loss="mean_squared_error", optimizer="adam")

        monitor = EarlyStopping(monitor="val_loss", min_delta=1e-3, patience=5, verbose=1, mode="auto")
        checkpointer = ModelCheckpoint(filepath=fileName, verbose=0, save_best_only=True)

        model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor, checkpointer], verbose=2, epochs=100)  
    
    model.load_weights(fileName)
    pred = model.predict(x_test)
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    results.append(score)
    print("Model: " + str(j) + "\nScore (RMSE): {}".format(score))
    print("R2:",metrics.r2_score(y_test,pred))
    print("MSE:", metrics.mean_squared_error(y_test, pred))
    step = step * step
    j += step
minValue = min(results)
minIndex = results.index(minValue) + 1
print("The best number of days was: " + str(minIndex)  + "\nThe RMSE was: " + str(minValue))

1
Epoch 1/100
79/79 - 2s - loss: 86000.5078 - val_loss: 672739.3125 - 2s/epoch - 25ms/step
Epoch 2/100
79/79 - 0s - loss: 62634.6172 - val_loss: 415647.0312 - 248ms/epoch - 3ms/step
Epoch 3/100
79/79 - 0s - loss: 38186.9219 - val_loss: 44522.5234 - 288ms/epoch - 4ms/step
Epoch 4/100
79/79 - 0s - loss: 17266.7188 - val_loss: 364461.0625 - 244ms/epoch - 3ms/step
Epoch 5/100
79/79 - 0s - loss: 8712.8877 - val_loss: 417223.1562 - 239ms/epoch - 3ms/step
Epoch 6/100
79/79 - 0s - loss: 6150.9443 - val_loss: 234733.9062 - 227ms/epoch - 3ms/step
Epoch 7/100
79/79 - 0s - loss: 4297.3828 - val_loss: 130485.1172 - 306ms/epoch - 4ms/step
Epoch 8/100
79/79 - 0s - loss: 3044.2412 - val_loss: 80006.0000 - 255ms/epoch - 3ms/step
Epoch 8: early stopping
Epoch 1/100
79/79 - 2s - loss: 85973.5156 - val_loss: 674502.2500 - 2s/epoch - 29ms/step
Epoch 2/100
79/79 - 0s - loss: 63246.0742 - val_loss: 405840.2500 - 275ms/epoch - 3ms/step
Epoch 3/100
79/79 - 0s - loss: 37698.4023 - val_loss: 38966.3477 - 305ms/e

KeyboardInterrupt: 